In [4]:
from spacy.tokens import Doc
import spacy
import random
import pandas as pd
import ast
from spacy.util import minibatch
from spacy.training.example import Example
import torch
from pdb import set_trace


In [35]:

print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [34]:
gpu = spacy.prefer_gpu()
gpu

True

In [ ]:

doc = Doc(nlp.vocab, words=["hello", "world", "!"], spaces=[True, False, False])

In [ ]:
for i in range(len(nlp.vocab)):
    print(nlp.vocab[i].text)

In [ ]:
help(Doc)

In [ ]:
ner_labels = nlp.get_pipe("ner").labels
ner_labels

In [ ]:
doc = nlp.make_doc("Apple is looking at buying U.K. startup for $1 billion")

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
other_pipes

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

In [ ]:
train_data = [
    ("What is the price of 10 bananas?", {"entities": [(21, 23, "QUANTITY"), (24, 31, "PRODUCT")]}),
    ("I need 3 bottles of water.", {"entities": [(7, 8, "QUANTITY"), (9, 22, "PRODUCT")]}),
    ("Can you get me 2 liters of milk?", {"entities": [(17, 18, "QUANTITY"), (19, 30, "PRODUCT")]}),
    ("Please buy 5 kg of rice.", {"entities": [(12, 13, "QUANTITY"), (14, 23, "PRODUCT")]}),
    ("He ordered 12 packs of noodles.", {"entities": [(11, 13, "QUANTITY"), (14, 29, "PRODUCT")]}),
    ("We have 20 apples in stock.", {"entities": [(8, 10, "QUANTITY"), (11, 17, "PRODUCT")]}),
    ("Pick up 1 dozen eggs from the store.", {"entities": [(8, 17, "QUANTITY"), (18, 22, "PRODUCT")]}),
    ("They bought 7 cans of soda.", {"entities": [(12, 13, "QUANTITY"), (14, 26, "PRODUCT")]}),
    ("There are 4 loaves of bread left.", {"entities": [(10, 11, "QUANTITY"), (12, 25, "PRODUCT")]}),
    ("Order 6 cartons of juice.", {"entities": [(6, 7, "QUANTITY"), (8, 23, "PRODUCT")]}),
    ("She took 9 bottles of olive oil.", {"entities": [(9, 10, "QUANTITY"), (11, 29, "PRODUCT")]}),
]




In [5]:
%cd /content/

[Errno 2] No such file or directory: '/content/'
/Users/senghok/Documents/Internship_2A/code/Spacy


In [7]:
synthethic_data = pd.read_csv("/Users/senghok/Documents/Internship_2A/code/gen_data/synthethic_data.csv")


In [8]:
synthethic_data

,sentence,entities
0,User 12845 accessed the server from IP 192.168...,"[(5, 10, 'CARDINAL'), (44, 57, 'IP-Address'), ..."
1,The backup completed at 15:30 on 2001:0db8:85a...,"[(32, 71, 'IP-Address'), (85, 117, 'Token keys..."
2,Payment of $2500 was processed on 10 April 202...,"[(12, 16, 'MONEY'), (36, 48, 'DATE'), (54, 63,..."
3,Server located in Paris (LOC) with IP 172.16.2...,"[(20, 25, 'LOC'), (36, 47, 'IP-Address'), (66,..."
4,Translator Maria used the language Spanish to ...,"[(11, 16, 'PER'), (28, 35, 'LANGUAGE'), (53, 7..."
...,...,...
1762,The server 'Alpha' at IP 203.0.113.31 logged 6...,"[(11, 16, 'WORK_OF_ART'), (20, 31, 'IP-Address..."
1763,Backup initiated by user John at 2024-06-02 fr...,"[(26, 30, 'PER'), (34, 44, 'DATE'), (50, 63, '..."
1764,User Mike accessed the system from IPv6 addres...,"[(5, 9, 'PER'), (40, 64, 'IP-Address'), (74, 1..."
1765,The document 'Advanced Networking' was edited ...,"[(13, 32, 'WORK_OF_ART'), (42, 52, 'DATE'), (6..."


In [15]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 99.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [16]:
# Load your base model
nlp = spacy.load("en_core_web_sm")


In [17]:

# Add or get NER pipe
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")



In [21]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
synthethic_data.head()

,sentence,entities
0,User 12845 accessed the server from IP 192.168...,"[(5, 10, 'CARDINAL'), (44, 57, 'IP-Address'), ..."
1,The backup completed at 15:30 on 2001:0db8:85a...,"[(32, 71, 'IP-Address'), (85, 117, 'Token keys..."
2,Payment of $2500 was processed on 10 April 202...,"[(12, 16, 'MONEY'), (36, 48, 'DATE'), (54, 63,..."
3,Server located in Paris (LOC) with IP 172.16.2...,"[(20, 25, 'LOC'), (36, 47, 'IP-Address'), (66,..."
4,Translator Maria used the language Spanish to ...,"[(11, 16, 'PER'), (28, 35, 'LANGUAGE'), (53, 7..."


In [28]:
# Add labels to the NER component
for annotations in synthethic_data["entities"]:
    annotations = ast.literal_eval(annotations)
    for ent in annotations:
        ner.add_label(ent[2])
        # ner.add_label(ent[2]) # The model will keep its old knowledge.

In [29]:
ner.labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'IP-Address',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MISC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PER',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'Token keys',
 'WORK_OF_ART')

In [31]:
# Disable other pipes to train only NER
other_pipes = [p for p in nlp.pipe_names if p not in ("transformer", "ner")]
other_pipes

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

### Convert to Spacy format

In [32]:
synthetic_data_spacy = []

for i in range(len(synthethic_data)):
    sentence, entity = synthethic_data.iloc[i]
    dict_entity = {}
    entity = ast.literal_eval(entity)
    dict_entity["entities"] = entity
    tup = (sentence,dict_entity)
    synthetic_data_spacy.append(tup)


In [33]:
synthetic_data_spacy[0]

('User 12845 accessed the server from IP 192.168.12.45 with token key 9f8d7a6b5c4e3d2f1a0b9c8d7e6f5a4b on 2024-04-12.',
 {'entities': [(5, 10, 'CARDINAL'),
   (44, 57, 'IP-Address'),
   (71, 103, 'Token keys'),
   (107, 117, 'DATE')]})

In [52]:
import spacy
from spacy.training import Example
import random
from spacy.util import minibatch
from spacy.training import offsets_to_biluo_tags

In [38]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.create_optimizer()  # Default Adam optimizer
    epochs = 10
    for epoch in range(epochs):
        random.shuffle(synthetic_data_spacy)
        losses = {}
        batches = minibatch(synthetic_data_spacy, size=8)  # Increased batch size for en_core_web_sm
        for batch in batches:
            examples = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)  # Create Example object
                examples.append(example)
            nlp.update(examples, drop=0.3, sgd=optimizer, losses=losses)
        print(f"Epoch {epoch+1}, Losses: {losses}")

/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "On 2023-11-11, technician Zoe reset the router 10...." with entities "[(3, 13, 'DATE'), (24, 27, 'PER'), (38, 48, 'IP-Ad...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "On July 4th, 2024, the server at IP fe80::abcd:123..." with entities "[(3, 14, 'DATE'), (30, 52, 'IP-Address'), (74, 110...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Epoch 1, Losses: {'ner': np.float32(1955.6206)}


KeyboardInterrupt: 

In [76]:
sentence, entities = synthetic_data_spacy[0]

In [81]:
entities = entities["entities"]

In [82]:
doc = nlp.make_doc(sentence)

In [83]:
biluo_tags = offsets_to_biluo_tags(doc, entities)

/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Client Miguel Hernandez connected from IPv4 8.8.8...." with entities "[(7, 23, 'PER'), (41, 46, 'IP-Address'), (60, 96, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [86]:
entities

[(7, 23, 'PER'),
 (41, 46, 'IP-Address'),
 (60, 96, 'Token keys'),
 (100, 110, 'DATE'),
 (118, 124, 'LOC')]

In [85]:
sentence

'Client Miguel Hernandez connected from IPv4 8.8.8.8 using token key ff00ee11dd22cc33bb44aa5566778899 on 2023-09-05 at the Madrid office.'

In [87]:
doc = nlp.make_doc(sentence)
for token in doc:
    print(f"Token: {token.text}, Start: {token.idx}, End: {token.idx + len(token.text)}")


Token: Client, Start: 0, End: 6
Token: Miguel, Start: 7, End: 13
Token: Hernandez, Start: 14, End: 23
Token: connected, Start: 24, End: 33
Token: from, Start: 34, End: 38
Token: IPv4, Start: 39, End: 43
Token: 8.8.8.8, Start: 44, End: 51
Token: using, Start: 52, End: 57
Token: token, Start: 58, End: 63
Token: key, Start: 64, End: 67
Token: ff00ee11dd22cc33bb44aa5566778899, Start: 68, End: 100
Token: on, Start: 101, End: 103
Token: 2023, Start: 104, End: 108
Token: -, Start: 108, End: 109
Token: 09, Start: 109, End: 111
Token: -, Start: 111, End: 112
Token: 05, Start: 112, End: 114
Token: at, Start: 115, End: 117
Token: the, Start: 118, End: 121
Token: Madrid, Start: 122, End: 128
Token: office, Start: 129, End: 135
Token: ., Start: 135, End: 136


In [88]:
entities = [(7, 23, 'PER'),
 (44, 51, 'IP-Address'),
 (68, 100, 'Token keys'),
 (104, 114, 'DATE'),
 (122, 128, 'LOC')]

In [89]:
biluo_tags = offsets_to_biluo_tags(doc, entities)

In [90]:
biluo_tags

['O',
 'B-PER',
 'L-PER',
 'O',
 'O',
 'O',
 'U-IP-Address',
 'O',
 'O',
 'O',
 'U-Token keys',
 'O',
 'B-DATE',
 'I-DATE',
 'I-DATE',
 'I-DATE',
 'L-DATE',
 'O',
 'O',
 'U-LOC',
 'O',
 'O']

In [63]:
TRAIN_DATA = ("XYZxyzg hat die beste Camera für Selfies", {"entities": [(0, 7, "BRAND"), (22, 28, "CAMERA")]})


In [64]:
sentence, entities= TRAIN_DATA

In [65]:
entities = entities["entities"]

In [66]:
doc = nlp.make_doc(sentence)

In [67]:
biluo_tags = offsets_to_biluo_tags(doc, entities)

In [68]:
biluo_tags

['U-BRAND', 'O', 'O', 'O', 'U-CAMERA', 'O', 'O']

In [54]:
biluo_tags

['O',
 'B-PER',
 'L-PER',
 'O',
 'O',
 '-',
 '-',
 'O',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'O',
 'O',
 'O',
 '-',
 '-',
 'O',
 'O']

In [ ]:
# Step 7: Save the model
nlp.to_disk("/content/fine_tuned_model")

# Step 8: Test the model
doc = nlp("Apple and xAI are companies.")
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
nlp.to_disk("custom_ner_model_trf")

trained_nlp_trf = spacy.load("custom_ner_model_trf")

/Applications/anaconda3/lib/python3.11/site-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map

In [ ]:

# Test
test_text =["How much for 3 oranges?","I want 15 chairs for the conference.", "Elon Musk is the CEO of Tesla"]
for text in test_text:
    doc = trained_nlp_trf(text)
    print(f"Text:{text}")
    print("Entities",[(ent.text,ent.label_) for ent in doc.ents])

Text:How much for 3 oranges?
Entities [('3', 'CARDINAL')]
Text:I want 15 chairs for the conference.
Entities [('15', 'CARDINAL')]
Text:Elon Musk is the CEO of Tesla
Entities [('Elon Musk', 'PERSON'), ('Tesla', 'ORG')]


In [ ]:
nlp = spacy.load("./custom_ner_model")